In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

False 2.2.0+cpu


In [1]:
from src import API
API.Trainer.main(stage = 0 , resume = 0 , checkname= 1)

24-06-23 20:26:13|MOD:display     |: Model Specifics:
24-06-23 20:26:13|MOD:display     |: Start Process [Data] at Sun Jun 23 20:26:13 2024!


--Process Queue : Data + Fit + Test
--Model_name is set to gru_dsize_day_ShortTest!
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=10) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=10,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=10) , display batch progress bar
Callback : StatusDisplay(verbosity=10) , display epoch and event information
Callback : ExportPredDF() , record and concat each model to Alpha model instance
Callback : SpecCB_DSize() , in _dsize model fill [size] in bat

24-06-23 20:26:15|MOD:display     |: Finish Process [Data], Cost 1.8 Secs
24-06-23 20:26:15|MOD:display     |: Start Process [Fit] at Sun Jun 23 20:26:15 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
 --> models blocks reading [risk_exp] DataBase...... finished! Cost 24.19 secs


Train Ep#  0 loss : 1.01989:   2%|▏         | 1/43 [00:00<00:11,  3.58it/s]

score function of [spearman] calculated and success!
loss function of [pearson] calculated and success!


Valid Ep#  0 score : 0.06677: 100%|██████████| 11/11 [00:01<00:00,  8.85it/s]
FirstBite Ep#  0 : loss  1.00527, train-0.00707, valid 0.02125, best 0.0213, lr1.3e-03
Valid Ep#  1 score : -0.03505: 100%|██████████| 11/11 [00:01<00:00,  9.21it/s]
FirstBite Ep#  1 : loss  0.98417, train 0.02161, valid-0.00164, best 0.0213, lr2.5e-03
Valid Ep#  2 score : -0.06946: 100%|██████████| 11/11 [00:01<00:00,  8.54it/s]
FirstBite Ep#  2 : loss  0.95409, train 0.05486, valid 0.02004, best 0.0213, lr3.8e-03
Valid Ep#  3 score : -0.08419: 100%|██████████| 11/11 [00:01<00:00,  6.66it/s]
FirstBite Ep#  3 : loss  0.92567, train 0.10738, valid 0.08075, best 0.0808, lr5.0e-03
24-06-23 20:27:45|MOD:display     |: gru_dsize_day_ShortTest #0 @20170103|FirstBite Ep#  4 Max Epoch|Train 0.1074 Valid 0.0808 BestVal 0.0808|Cost  1.5Min, 17.5Sec/Ep
24-06-23 20:27:45|MOD:display     |: Finish Process [Fit], Cost 0.0 Hours, 1.5 Min/model, 22.4 Sec/Epoch
24-06-23 20:27:45|MOD:display     |: Start Process [Test] at Sun 

Group optimization of 3 alphas , 3 benchmarks , 3 lags , 35 dates , (945 opts) start!
Done Optimize [0_best_0.csi300      ] at 20170104 , time cost (ms) : {'parse_input': 14.96, 'solve': 72.8, 'output': 15.92}
Done Optimize [0_best_0.csi500      ] at 20170104 , time cost (ms) : {'parse_input': 15.0, 'solve': 51.86, 'output': 15.92}
Done Optimize [0_best_0.csi1000     ] at 20170104 , time cost (ms) : {'parse_input': 15.0, 'solve': 51.85, 'output': 15.96}
Done Optimize [0_best_0.csi300.1    ] at 20170104 , time cost (ms) : {'parse_input': 13.96, 'solve': 58.81, 'output': 15.96}
Done Optimize [0_best_0.csi500.1    ] at 20170104 , time cost (ms) : {'parse_input': 13.99, 'solve': 50.88, 'output': 15.92}
Done Optimize [0_best_0.csi1000.1   ] at 20170104 , time cost (ms) : {'parse_input': 13.96, 'solve': 49.87, 'output': 16.0}
Done Optimize [0_best_0.csi300.2    ] at 20170104 , time cost (ms) : {'parse_input': 13.97, 'solve': 56.84, 'output': 15.92}
Done Optimize [0_best_0.csi500.2    ] at 20

AllTimeAvg   0.0614  0.0614  0.0845
AllTimeSum     2.15    2.15    2.96
Std          0.0793  0.0793  0.1478
TValue         4.58    4.58    3.38
AnnIR        3.7964  3.7964  2.8023
24-06-23 20:32:13|MOD:display     |: Finish Process [Test], Cost 268.2 Secs
24-06-23 20:32:13|MOD:time        |: Main Process Finished! Cost 5 Minutes 59.7 Seconds


   factor_name benchmark     start       end        pf        bm    excess  \
0     0_best_0   csi1000  20170105  20170308 -0.067261 -0.009277 -0.057984   
1     0_best_0    csi300  20170105  20170308 -0.034304  0.023920 -0.058224   
2     0_best_0    csi500  20170105  20170308 -0.057443  0.016063 -0.073507   
3  0_swabest_0   csi1000  20170105  20170308 -0.036647 -0.009277 -0.027370   
4  0_swabest_0    csi300  20170105  20170308 -0.041410  0.023920 -0.065331   
5  0_swabest_0    csi500  20170105  20170308 -0.043192  0.016063 -0.059256   
6  0_swalast_0   csi1000  20170105  20170308 -0.067267 -0.009277 -0.057989   
7  0_swalast_0    csi300  20170105  20170308 -0.034302  0.023920 -0.058222   
8  0_swalast_0    csi500  20170105  20170308 -0.057442  0.016063 -0.073505   

   annualized       mdd        te        ir    calmar   turnover  \
0   -0.298407  0.061549  0.040601 -7.349663 -4.848265  14.600001   
1   -0.292199  0.055738  0.033370 -8.756379 -5.242379  14.600001   
2   -0.356858  

In [16]:
import numpy as np

def winsorize_by_bnded_dist(src_: np.ndarray, bnd_: float = 0.25, m_: float = 3.5, dist_type_: int=0):
    # dist_type_:0, normal
    # dist_type_:1, log_normal
    assert bnd_ > 0 and bnd_ < 50
    assert src_.ndim == 1
    assert m_ > 0
    assert dist_type_ in (0, 1)
    if dist_type_ == 1:
        assert np.all(np.logical_or(np.isnan(src_), src_ > 0)), "winsorize_by_dist>>src_ must be positive when dist type as 1(log_normal)"
        des = np.log(src_)
    else:
        des = src_.copy()
    lu = np.nanpercentile(des, [bnd_, 100 - bnd_])
    bnded_flg = np.logical_and(des >= lu[0], des <= lu[1])
    avg = np.nanmean(des[bnded_flg])
    std = np.nanstd(des[bnded_flg])
    ub = avg + m_ * std
    lb = avg - m_ * std
    des[des > ub] = np.nan
    des[des < lb] = np.nan
    if dist_type_ == 1:
        des = np.exp(des)
    return des

winsorize_by_bnded_dist(np.random.randn(100),0.05,2)

array([ 1.07445354,  0.82797318,  0.40891351, -0.04522681, -1.2157853 ,
       -0.36239632,  0.73562442, -0.80452426, -0.02551805,  1.28800358,
        1.18027524, -0.64574498,  0.80558861, -0.71658249,  1.51474782,
       -0.62979321,  0.25221309,  0.57496398,  0.71905066,  0.28415118,
       -0.77847613,  0.8542863 , -0.00677902, -1.62531215,  0.80277862,
        1.09889713, -0.55343213,  0.22747532, -1.10893855,  1.49321991,
        1.34694109,         nan, -1.07385299,  0.79543078, -0.92405813,
       -0.43991641,  0.86919155, -0.41860346, -0.16922205,  1.6347842 ,
       -0.35679995, -0.96212819, -0.99812774, -0.44819177, -0.5923116 ,
       -1.29085336,         nan, -1.60465504,  0.78318216,  0.48820411,
       -1.82439061, -0.49540753, -0.18168658,         nan, -0.04888148,
       -0.206641  ,  0.58717214, -1.13665894, -1.30008417,  0.55137061,
       -0.2736447 ,         nan,  0.10927408,  0.09780903,  0.37858274,
        1.39094663, -0.15564593, -0.98810931,  0.40040585,  0.63

In [13]:
#csi800 , csi1000
import os
import numpy as np
import pandas as pd
from src.factor.basic import Port

def csi800(date):
    path = f'D:/Coding/ChinaShareModel/ModelData/B_index_weight/1_csi_index/CSI800/CSI800_{date}.csv'
    if not os.path.exists(path): raise ValueError(date)
    with np.errstate(invalid='ignore' , divide = 'ignore'):
        df = pd.read_csv(path)
        df = adjust_secid(df)
        df['weight'] = df['weight'] / df['weight'].sum()
        df = adjust_precision(df)
    return df

def csi1000_1(date):
    path = f'D:/Coding/ChinaShareModel/ModelData/B_index_weight/1_csi_index/CSI1000/CSI1000_{date}.csv'
    if not os.path.exists(path): raise ValueError(date)
    with np.errstate(invalid='ignore' , divide = 'ignore'):
        df = pd.read_csv(path)
        df = adjust_secid(df)
        df['weight'] = df['weight'] / df['weight'].sum()
        df = adjust_precision(df)
    return df

def csi1000_2(date):
    path = f'D:/Coding/ChinaShareModel/ModelData/B_index_weight/2_wind_database/CSI1000/CSI1000_{date}.csv'
    if not os.path.exists(path): raise ValueError(date)
    with np.errstate(invalid='ignore' , divide = 'ignore'):
        df = pd.read_csv(path)
        df = adjust_secid(df)
        df['weight'] = df['weight'] / df['weight'].sum()
        df = adjust_precision(df)
    return df

def adjust_secid(df : pd.DataFrame):
    '''switch old wind_id into secid'''
    if 'wind_id' not in df.columns.values: return df

    df['wind_id'] = df['wind_id'].astype(str)
    replace_dict = {'T00018' : '600018'}
    df['wind_id'] = df['wind_id'].str.slice(0, 6).replace(replace_dict)
    df['wind_id'] = df['wind_id'].where(df['wind_id'].str.isdigit() , '-1')
    df['wind_id'] = df['wind_id'].astype(int)
    return df.rename(columns={'wind_id':'secid'})

def adjust_precision(df : pd.DataFrame , tol = 1e-8 , dtype_float = np.float32 , dtype_int = np.int64):
    '''adjust precision for df columns'''
    for col in df.columns:
        if np.issubdtype(df[col].to_numpy().dtype , np.floating): 
            df[col] = df[col].astype(dtype_float)
            df[col] *= (df[col].abs() > tol)
        if np.issubdtype(df[col].to_numpy().dtype , np.integer): 
            df[col] = df[col].astype(dtype_int)
    return df

path = f'D:/Coding/ChinaShareModel/ModelData/B_index_weight/1_csi_index/CSI800'
d800 = [p.split('_')[-1].split('.')[0] for p in os.listdir(path)]
path2 = f'D:/Coding/ChinaShareModel/ModelData/B_index_weight/1_csi_index/CSI1000'
path3 = f'D:/Coding/ChinaShareModel/ModelData/B_index_weight/2_wind_database/CSI1000'
d1000_tar = [d for d in d800 if int(d) >= 20150529]
d1000_csi = np.array([p.split('_')[-1].split('.')[0] for p in os.listdir(path2)]).astype(int)
d1000_wind = np.array([p.split('_')[-1].split('.')[0] for p in os.listdir(path3)]).astype(int)
for d in d1000_tar:
    tar_path = f'data/DataBase/DB_benchmark/csi1000/{d[:4]}/csi1000.{d}.feather'
    if os.path.exists(tar_path): continue
    if d in d1000_csi:
        df = csi1000_1(d)
    else:
        d_near_1 = d1000_csi[d1000_csi <= int(d)].max() if d1000_csi.min() <= int(d) else -1
        d_near_2 = d1000_wind[d1000_wind <= int(d)].max()
        if d_near_1 >= d_near_2:
            d_use = d_near_1 
            pf = csi1000_1(d_use)
        else:
            d_use = d_near_2
            df = csi1000_2(d_use)
        p = Port(df , d_use).evolve_to_date(int(d))
        df = p.port
    os.makedirs(os.path.dirname(tar_path) , exist_ok=True)
    df.to_feather(tar_path)
    print(d)

20150529
20150601
20150602
20150603
20150604
20150605
20150608
20150609
20150610
20150611
20150612
20150615
20150616
20150617
20150618
20150619
20150623
20150624
20150625
20150626
20150629
20150630
20150701
20150702
20150703
20150706
20150707
20150708
20150709
20150710
20150713
20150714
20150715
20150716
20150717
20150720
20150721
20150722
20150723
20150724
20150727
20150728
20150729
20150730
20150731
20150803
20150804
20150805
20150806
20150807
20150810
20150811
20150812
20150813
20150814
20150817
20150818
20150819
20150820
20150821
20150824
20150825
20150826
20150827
20150828
20150831
20150901
20150902
20150907
20150908
20150909
20150910
20150911
20150914
20150915
20150916
20150917
20150918
20150921
20150922
20150923
20150924
20150925
20150928
20150929
20150930
20151008
20151009
20151012
20151013
20151014
20151015
20151016
20151019
20151020
20151021
20151022
20151023
20151026
20151027
20151028
20151029
20151030
20151102
20151103
20151104
20151105
20151106
20151109
20151110
20151111
2

In [12]:
d1000_csi[d1000_csi <= int(d)]

array([], dtype=int32)

In [7]:
d800

['20041231',
 '20050104',
 '20050105',
 '20050106',
 '20050107',
 '20050110',
 '20050111',
 '20050112',
 '20050113',
 '20050114',
 '20050117',
 '20050118',
 '20050119',
 '20050120',
 '20050121',
 '20050124',
 '20050125',
 '20050126',
 '20050127',
 '20050128',
 '20050131',
 '20050201',
 '20050202',
 '20050203',
 '20050204',
 '20050216',
 '20050217',
 '20050218',
 '20050221',
 '20050222',
 '20050223',
 '20050224',
 '20050225',
 '20050228',
 '20050301',
 '20050302',
 '20050303',
 '20050304',
 '20050307',
 '20050308',
 '20050309',
 '20050310',
 '20050311',
 '20050314',
 '20050315',
 '20050316',
 '20050317',
 '20050318',
 '20050321',
 '20050322',
 '20050323',
 '20050324',
 '20050325',
 '20050328',
 '20050329',
 '20050330',
 '20050331',
 '20050401',
 '20050404',
 '20050405',
 '20050406',
 '20050407',
 '20050408',
 '20050411',
 '20050412',
 '20050413',
 '20050414',
 '20050415',
 '20050418',
 '20050419',
 '20050420',
 '20050421',
 '20050422',
 '20050425',
 '20050426',
 '20050427',
 '20050428',

In [1]:
import pandas as pd
df = pd.read_feather('model/gru_dsize_day_ShortTest/pred_df.feather')
df

,model_num,model_type,output_num,secid,date,value
0,2,best,0,1,20161123,-0.022257
1,2,best,0,2,20161123,-1.571142
2,2,best,0,4,20161123,-1.659993
3,2,best,0,5,20161123,0.512646
4,2,best,0,6,20161123,0.608763
...,...,...,...,...,...,...
2812,2,swabest,1,603993,20170228,-1.841711
2813,2,swabest,1,603996,20170228,-0.594857
2814,2,swabest,1,603997,20170228,-0.363169
2815,2,swabest,1,603998,20170228,0.538781


In [3]:
df.pivot_table('value',['secid','date'],'model_type')

model_type           best   swabest   swalast
secid  date                                  
1      20161123 -0.023518  0.023127  0.354941
       20161124 -0.598424 -0.323617 -0.155266
       20161125 -1.010154 -0.424851 -0.520247
       20161128 -1.425906 -0.920963 -0.888758
       20161129 -1.225299 -0.270630 -0.710745
...                   ...       ...       ...
603999 20170222 -0.384051  0.119342  0.034980
       20170223 -0.829171 -0.266512 -0.359622
       20170224 -0.504662  0.311532 -0.071804
       20170227 -1.320969 -0.520519 -0.795831
       20170228 -1.742621 -1.632039 -1.169885

[177208 rows x 3 columns]

In [1]:
from src import API
API.DataAPI.reconstruct_train_data()
API.Trainer.update_models()

Use device name: NVIDIA GeForce RTX 4090
predict is False , Data Processing start!
4 datas :['y', 'day', '30m', 'risk']
y blocks loading start!
 --> labels blocks reading [ret10_lag] DataBase...... finished! Cost 20.05 secs
 --> labels blocks reading [ret20_lag] DataBase...... finished! Cost 18.36 secs
 --> labels blocks merging (2)...... finished! Cost 2.97 secs
 --> models blocks reading [risk_exp] DataBase...... finished! Cost 66.12 secs
y blocks loading finished! Cost 118.05 secs
y blocks process...... finished! Cost 45.72 secs
y blocks masking...... finished! Cost 0.81 secs
y blocks saving ...... finished! Cost 4.98 secs
y blocks norming...... finished! Cost 0.00 secs
y finished! Cost 169.72 Seconds
--------------------------------------------------------------------------------
day blocks loading start!
 --> trade blocks reading [day] DataBase...... finished! Cost 30.05 secs
day blocks loading finished! Cost 30.08 secs
day blocks process...... finished! Cost 3.38 secs
day blocks 